# Welcome to MESSAGEix

## Integrated Assessment Modeling for the 21st Century

### Matthew Gidden, International Institute for Applied Systems Analysis

#### Email: gidden@iiasa.ac.at, Github: @gidden

#### https://iiasa.github.io/ene-present.github.io/cnrs-2018

# Building a Simple Energy Model

The goal of this tutorial is to build a simple energy model using `MESSAGEix` with minimal features that can be expanded in future tutorials.  

We will build the model component by component, focusing on both the **how** (code implementation) and **why** (mathematical formulation).

# Follow Along at Home

The full model documentation is available online at [https://messageix.iiasa.ac.at](https://messageix.iiasa.ac.at)
    
<img src='assests/doc_page.png'>

And you can easily install `MESSAGEix` yourself and get all the tutorials:

```shell
$ conda install -c conda-forge message-ix
$ messageix-dl # install all tutorials to your current directory
```

# MESSAGEix

<img src='assests/austria.png' width='700'>

# MESSAGEix: the Model

At it's core, an optimization problem:

$\min c^T x$  
$s.t. Ax \leq b$  

- system of **constraint** equations
- optimizing an **objective function**, nominally minimizing total system **cost**
- decision VARIABLES ($x$) are capitalized
- parameters ($A$, $b$) are lower case

## MESSAGEix: the Tool

<img src='assests/message.png' width='700'>

# A Brave New World: Westeros with Electricity!

<table align='center'><tr><td><img src='assests/westeros.jpg' width='150'></td><td><img src='assests/base_res.png'></td></tr></table>

## Setup

First, we import all the packages we need.

In [ ]:
from __future__ import division # makes division work the same way between py2 and py3

In [ ]:
import pandas as pd
import ixmp as ix
import message_ix

from message_ix.utils import make_df

%matplotlib inline

The `Platform` is your connection to a database

In [ ]:
mp = ix.Platform(dbtype='HSQLDB')

Once connected, we make our `Scenario` which we use to build our model.

In [ ]:
scenario = message_ix.Scenario(mp, model='Westeros Electrified', 
                               scenario='baseline', version='new')

## Model Structure

We start by defining basic characteristics of the model, including time, space, and the energy system structure.

The model horizon will span 3 decades. Let's assume that we're far in the future after the events of A Song of Ice and Fire (which occur ~300 years after Aegon the conqueror).

| Math Notation | Model Meaning                |
|---------------|------------------------------|
| $y \in Y^H$   | time periods in history      |
| $y \in Y^M$   | time periods in model horizon|

In [ ]:
history = [690]
model_horizon = [700, 710, 720]
scenario.add_horizon({'year': history + model_horizon,  
                      'firstmodelyear': model_horizon[0]})

Our model will have a single `node`, i.e., its spatial dimension.


| Math Notation | Model Meaning|
|---------------|--------------|
| $n \in N$     | node         |

In [ ]:
country = 'Westeros'
scenario.add_spatial_sets({'country': country})

And we fill in the energy system's `commodities`, `levels`, `technologies`, and `mode` (defining how certain technologies operate). 


| Math Notation | Model Meaning|
|---------------|--------------|
| $c \in C$     | commodity    |
| $l \in L$     | level        |
| $t \in T$     | technology   |
| $m \in M$     | mode         |

In [ ]:
scenario.add_set("commodity", ["electricity", "light"])

scenario.add_set("level", ["secondary", "final", "useful"])

scenario.add_set("technology", ['coal_ppl', 'wind_ppl', 'grid', 'bulb'])

scenario.add_set("mode", "standard")

## Supply and Demand (or Balancing Commodities)

The fundamental premise of the model is to satisfy demand. To first order, demand for services like electricity track with economic productivity (GDP). We define a GDP profile similar to first-world GDP growth from [1900-1930](https://en.wikipedia.org/wiki/List_of_regions_by_past_GDP):

In [ ]:
gdp_profile = pd.Series([1., 1.5, 1.9], 
                        index=pd.Index(model_horizon, name='Time'))
gdp_profile.plot(title='Demand')

The `COMMODITY_BALANCE` equation ensures that `demand` for each `commodity` is met at each `level` in the energy system.

$\sum_{\substack{n^L,t,m \\ y^V \leq y}} output_{n^L,t,y^V,y,m,n,c,l} \cdot ACT_{n^L,t,y^V,y,m}$  
$- \sum_{\substack{n^L,t,m, \\ y^V \leq y}} input_{n^L,t,y^V,y,m,n,c,l} \cdot ACT_{n^L,t,m,y}$  
$\geq demand_{n,c,l,y} \quad \forall \ l \in L$

While `demand` must be met, it can also be *exceeded* allowing the model to plan for future periods of demand for storable commodities.


First we establish demand. Let's assume

- 40 million people in [300 AC](https://atlasoficeandfireblog.wordpress.com/2016/03/06/the-population-of-the-seven-kingdoms/)
- similar population growth to Earth in the same time frame [(~factor of 12)](https://en.wikipedia.org/wiki/World_population_estimates)
- a per capita demand for electricity of 1000 kWh
- and 8760 hours in a year (of course!)

Then we can add the demand parameter

Note present day: [~17000 GWh in Austria](http://www.iea.org/statistics/statisticssearch/report/?country=AUSTRIA&product=electricityandheat&year=2015) with population [~9M](http://www.austria.org/population/) which is ~1890 kWh per capita

In [ ]:
demand_per_year = 40 * 12 * 1000 / 8760
light_demand = pd.DataFrame({
        'node': country,
        'commodity': 'light',
        'level': 'useful',
        'year': model_horizon,
        'time': 'year',
        'value': (100 * gdp_profile).round(),
        'unit': 'GWa',
    })

In [ ]:
scenario.add_par("demand", light_demand)

In [ ]:
#TODO Hide this one

year_df = scenario.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']

base_input = {
    'node_loc': country,
    'year_vtg': vintage_years
    ,
    'year_act': act_years,
    'mode': 'standard',
    'node_origin': country,
    'commodity': 'electricity',
    'time': 'year',
    'time_origin': 'year',
}

base_output = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'node_dest': country,
    'time': 'year',
    'time_dest': 'year', 
    'unit': '%',
}

Working backwards along the Reference Energy System, we can add connections for the `bulb`

In [ ]:
bulb_out = make_df(base_output, technology='bulb', commodity='light', 
                   level='useful', value=1.0)
scenario.add_par('output', bulb_out)

bulb_in = make_df(base_input, technology='bulb', commodity='electricity',  
                  level='final', value=1.0, unit='%')
scenario.add_par('input', bulb_in)

Next, the `grid`, with loses of 13%

In [ ]:
grid_efficiency = 1
grid_out = make_df(base_output, technology='grid', commodity='electricity', 
                   level='final', value=grid_efficiency)
scenario.add_par('output', grid_out)

grid_in = make_df(base_input, technology='grid', commodity='electricity',
                  level='secondary', value=1.0, unit='%')
scenario.add_par('input', grid_in)

And finally, our power plants. The model does not include the fossil resources used as `input` for coal plants; however, costs of coal extraction are included in the parameter $variable\_cost$.

In [ ]:
coal_out = make_df(base_output, technology='coal_ppl', commodity='electricity', 
                   level='secondary', value=1.)
scenario.add_par('output', coal_out)

wind_out = make_df(base_output, technology='wind_ppl', commodity='electricity', 
                   level='secondary', value=1.)
scenario.add_par('output', wind_out)

## Operational Constraints and Parameters

The model has a number of "reality" constraints, which relate built capacity to available power.

The `CAPACITY_CONSTRAINT` 

$\sum_{m} ACT_{n,t,y^V,y,m,h}
    \leq duration\_time_{h} \cdot capacity\_factor_{n,t,y^V,y,h} \cdot CAP_{n,t,y^V,y}
    \quad t \ \in \ T^{INV}$


This requires us to provide capacity factors

In [ ]:
#TODO: Hide this
base_capacity_factor = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'time': 'year',
    'unit': '%',
}

In [ ]:
capacity_factor = {
    'coal_ppl': 1,
    'wind_ppl': 1,
    'bulb':1, 
}

for tec, val in capacity_factor.items():
    df = make_df(base_capacity_factor, technology=tec, value=val)
    scenario.add_par('capacity_factor', df)

The model can further be provided `technical_lifetime`s in order to properly manage deployed capacity and related costs via the `CAPACITY_MAINTENENCE` constraint:

$CAP_{n,t,y^V,y} \leq remaining\_capacity_{n,t,y^V,y} \cdot value \: \: \forall t \in T^{INV}$

Where `value` can take different forms depending on what time period is considered:

| Value                                 | Condition                                                      |
|---------------------------------------|----------------------------------------------------------------|
| $\Delta_y historical\_new\_capacity_{n,t,y^V}$ | $y$ is first model period                                      |
| $\Delta_y CAP\_NEW_{n,t,y^V}$                  | $y = y^V$                                                      |
| $CAP_{n,t,y^V,y-1}$                   | if $y > y^V$ and $y - y^V < technical\_lifetime_{n,t,y^V}$     |


In [ ]:
base_technical_lifetime = {
    'node_loc': country,
    'year_vtg': model_horizon,
    'unit': 'y',
}

In [ ]:
lifetime = {
    'coal_ppl': 20,
    'wind_ppl': 20,
    'bulb': 1,
}

for tec, val in lifetime.items():
    df = make_df(base_technical_lifetime, technology=tec, value=val)
    scenario.add_par('technical_lifetime', df)

## Technological Diffusion and Contraction

We know from historical precedent that energy systems can not be transformed instantaneously. Therefore, we use a family of constraints on **activity** (`ACT`) and **capacity** (`CAP`). 

$\sum_{y^V \leq y,m} ACT_{n,t,y^V,y,m,h} \leq$  
$initial\_activity\_up_{n,t,y,h}
        \cdot \frac{ \Big( 1 + growth\_activity\_up_{n,t,y,h} \Big)^{|y|} - 1 }
                   { growth\_activity\_up_{n,t,y,h} }+ \Big( 1 + growth\_activity\_up_{n,t,y,h} \Big)^{|y|} \cdot \Big( \sum_{y^V \leq y-1,m} ACT_{n,t,y^V,y-1,m,h} + \sum_{m} historical\_activity_{n,t,y-1,m,h}\Big)$  

This example limits the ability for technologies to **grow**. To do so, we  need to provide `growth_activity_up` values for each technology that we want to model as being diffusion constrained. Here, we set this constraint at 5% per year.

In [ ]:
## TODO: hide this
base_growth = {
    'node_loc': country,
    'year_act': model_horizon,
    'time': 'year',
    'unit': '%',
}

In [ ]:
growth_technologies = [
    "coal_ppl", 
    "wind_ppl", 
]

for tec in growth_technologies:
    df = make_df(base_growth, technology=tec, value=0.1) 
    scenario.add_par('growth_activity_up', df)

## Defining an Energy Mix

To model the transition of an energy system, one must start with the existing system which are defined by the parameters `historical_activity` and `historical_capacity`. These parameters define the energy mix before the model horizon.  

We begin by defining a few key values:

- how much useful energy was needed
- how much final energy was generated
- and the mix for different technologies

In [ ]:
historic_demand = 0.85 * demand_per_year
historic_generation = historic_demand / grid_efficiency
coal_fraction = 0.6

In [ ]:
# TODO: hide

base_capacity = {
    'node_loc': country,
    'year_vtg': history,
    'unit': 'GWa',
}

base_activity = {
    'node_loc': country,
    'year_act': history,
    'mode': 'standard',
    'time': 'year',
    'unit': 'GWa',
}


Then, we can define the **activity** and **capacity** in the historic period

In [ ]:
old_activity = {
    'coal_ppl': coal_fraction * historic_generation,
    'wind_ppl': (1 - coal_fraction) * historic_generation,
}

for tec, val in old_activity.items():
    df = make_df(base_activity, technology=tec, value=val)
    scenario.add_par('historical_activity', df)

In [ ]:
act_to_cap = {
    'coal_ppl': 1 / 10 / capacity_factor['coal_ppl'] / 2, # 20 year lifetime
    'wind_ppl': 1 / 10 / capacity_factor['wind_ppl'] / 2,
}

for tec in act_to_cap:
    value = old_activity[tec] * act_to_cap[tec]
    df = make_df(base_capacity, technology=tec, value=value)
    scenario.add_par('historical_new_capacity', df)

## Objective Function

The objective function drives the purpose of the optimization. Do we wish to seek maximum utility of the social planner, minimize carbon emissions, or something else? Classical IAMs seek to minimize total discounted system cost over space and time. 

$\min \sum_{n,y \in Y^{M}} interestrate_{y} \cdot COST\_NODAL_{n,y}$


First, let's add the interest rate parameter.

In [ ]:
rate = [0.05] * len(model_horizon)
unit = ['%'] * len(model_horizon)
scenario.add_par("interestrate", key=model_horizon, val=rate, unit=unit)

`COST_NODAL` is comprised of a variety of costs related to the use of different technologies.

### Investment Costs

Capital, or investment, costs are invoked whenever a new plant or unit is built

$inv\_cost_{n,t,y} \cdot construction\_time\_factor_{n,t,y} \cdot CAP\_NEW_{n,t,y}$

In [ ]:
# TODO hide this
base_inv_cost = {
    'node_loc': country,
    'year_vtg': model_horizon,
    'unit': 'USD/GWa',
}

In [ ]:
# in $ / kW
costs = {
    'coal_ppl': 500,
    'wind_ppl': 1500,
    'bulb': 5,
}

for tec, val in costs.items():
    df = make_df(base_inv_cost, technology=tec, value=val)
    scenario.add_par('inv_cost', df)

### Fixed O&M Costs

Fixed cost are only relevant as long as the capacity is active. This formulation allows to include the potential cost savings from early retirement of installed capacity.

$\sum_{y^V \leq y} \ fix\_cost_{n,t,y^V,y} \cdot CAP_{n,t,y^V,y}$

In [ ]:
# TODO Hide
base_fix_cost = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'unit': 'USD/GWa',
}


In [ ]:
# in $ / kW
costs = {
    'coal_ppl': 30,
    'wind_ppl': 10,
}

for tec, val in costs.items():
    df = make_df(base_fix_cost, technology=tec, value=val)
    scenario.add_par('fix_cost', df)

### Variable O&M Costs

Variable Operation and Maintence costs are associated with the costs of actively running the plant. Thus, they are not applied if a plant is on standby (i.e., constructed, but not currently in use).

$\sum_{\substack{y^V \leq y \\ m,h}} \ var\_cost_{n,t,y^V,y,m,h} \cdot ACT_{n,t,y^V,y,m,h} $

In [ ]:
# TODO hide
base_var_cost = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'time': 'year',
    'unit': 'USD/GWa',
}

In [ ]:
# in $ / MWh
costs = {
    'coal_ppl': 30,
    'grid': 50,
}

for tec, val in costs.items():
    df = make_df(base_var_cost, technology=tec, value=val)
    scenario.add_par('var_cost', df)

A full model will also have costs associated with

- resource extraction: $\sum_{c,g} \ resource\_cost_{n,c,g,y} \cdot EXT_{n,c,g,y} $
- emissions (TODO recopy this, hats no good): $\sum_{e,t, e \in E(e)} emission\_scaling_{e,e} \cdot \ emission\_tax_{n,e,t,y} \cdot EMISS_{n,e,t,y}$
- land use (emulator): $\sum_{s} land\_cost_{n,s,y} \cdot LAND_{n,s,y}$
- exceedence on "hard" bounds of technology expansion and contraction: $\sum_{m,h} \ \Big( abs\_cost\_activity\_soft\_lo_{n,t,y,m,h} + level\_cost\_activity\_soft\_lo_{n,t,y,m,h} \cdot\ levelized\_cost_{n,t,y,m,h}\Big) \cdot ACT\_LO_{n,t,y,h}$

## Time to Solve the Model

In [ ]:
scenario.commit(comment='basic model of Westerosi electrification')
scenario.set_as_default()

In [ ]:
scenario.solve()

In [ ]:
scenario.var('OBJ')['lvl']

# Plotting Results

In [ ]:
from tools import Plots
p = Plots(scenario, country, firstyear=model_horizon[0])

# Activity

How much energy is generated in each time period from the different potential sources?

In [ ]:
p.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

## Capacity

Given how many new plants are built, how many are actually used?

In [ ]:
p.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

## Electricity Price

And how much does the electricity cost? These prices are taken from the **dual variables** of the solution and are given the name **shadow prices**. They reflect the marginal price of electricity, taken from the most expensive producer.  

Note that the price drop when the most expensive technology is no longer in the system.

In [ ]:
p.plot_prices(subset=['light'], baseyear=True)

# Fin

With that, you have built and run your very first MESSAGEix model. Welcome to the community!

Check us out on Github: https://github.com/iiasa/message_ix  

Get in touch with us online: https://groups.google.com/forum/message-ix  

And feel free to contact me with any further questions: gidden@iiasa.ac.at

In [ ]:
mp.close_db()